In [ ]:
import json
import ipdb
from vllm import LLM, SamplingParams
import json
import os
model_name = "/data/ah/code/rl/open-rs/results/pararel"
model_name = "/data/ah/code/rl/open-rs/results/qwen1.5b_pararel_origin_reward_fix_unreward/checkpoint-600"
model_name = "/data/ah/code/rl/open-rs/results/llama3/pararel_batch10_51_2"
model_name = "/data/ah/code/rl/open-rs/results/llama3/pararel_batch10"
model_name = "results/llama3/pararel_batch10_53_strict_contents_format8"
model_name = "results/llama3/pararel_batch10_53_strict_answer_se_1"
model_name = "results/qwen/pararel_batch10_strict_answer_se_1"
model_name = "results/qwen/pararel_batch10_strict_answer_origin_data"
model_name = "results/qwen/pararel_batch10_strict_answer_origin_data_0.7temp"
# cuda :1
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
model = LLM(
    model=model_name,
)
print("model load sucess")


INFO 05-07 02:16:40 __init__.py:190] Automatically detected platform cuda.
INFO 05-07 02:16:48 config.py:542] This model supports multiple tasks: {'classify', 'generate', 'embed', 'score', 'reward'}. Defaulting to 'generate'.
INFO 05-07 02:16:48 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='results/qwen/pararel_batch10_strict_answer_origin_data_0.7temp', speculative_config=None, tokenizer='results/qwen/pararel_batch10_strict_answer_origin_data_0.7temp', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_t

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 05-07 02:16:51 model_runner.py:1115] Loading model weights took 2.8875 GB
INFO 05-07 02:16:52 worker.py:267] Memory profiling takes 1.16 seconds
INFO 05-07 02:16:52 worker.py:267] the current vLLM instance can use total_gpu_memory (47.29GiB) x gpu_memory_utilization (0.90) = 42.56GiB
INFO 05-07 02:16:52 worker.py:267] model weights take 2.89GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 37.57GiB.
INFO 05-07 02:16:53 executor_base.py:110] # CUDA blocks: 87931, # CPU blocks: 9362
INFO 05-07 02:16:53 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 42.94x
INFO 05-07 02:16:55 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes:   9%|▊         | 3/35 [00:01<00:16,  1.99it/s]

CUDA_VISIBLE_DEVICES=1,2,4,5,6 ACCELERATE_LOG_LEVEL=info accelerate launch   --config_file recipes/accelerate_configs/zero3.yaml   --num_processes=5   src/open_r1/pararel_grpo.py   --config  recipes/llama/llama_pararel_batch10_53_strict_contents_format8.yaml

In [2]:
# from src.open_r1.rewards_u import extract_content, are_equivalent
from tqdm import tqdm
import os
import re
import math

# 定义采样参数
sampling_params = SamplingParams(
    temperature=0.1,
    max_tokens=40
)

def extract_content(text, tag_type="answer"):
    if tag_type == "answer":
        pattern = r"<answer>(.*?)</answer>"
    else:  # confidence
        pattern = r"<confidence>(.*?)</confidence>"
        # confidence = text.split("<confidence>")[-1]
        # confidence = confidence.split("</confidence>")[0]
        # return confidence.strip()

    matches = re.findall(pattern, text, re.DOTALL)
    return matches[0].strip() if matches else ""

data_path = "dataset/ID_test_pararel.json"
# data_path = "dataset/OOD_test_pararel.json"
# data_path = "dataset/pararel_training.json"
dataset = json.load(open(data_path))

def inference(input_text):
    outputs = model.generate(
            input_text,
            sampling_params=sampling_params
    )
    
    output_text = outputs[0].outputs[0].text
    # 找到第一个</confidence>， 保留第一个</confidence>及其之前的文本
    result = output_text.split('</confidence>')[0] + '</confidence>'
    # print(result)
    return result

sys_prompt = """
  You are a faithful assistant. The user asks a question, and you provide the answer and your confidence expressed in sure or unsure.
  Question: Who is representing The Stranglers? 
  Answer: <answer>EMI</answer>
  <confidence>sure</confidence>

  Question: Who is the president of the Shao Qiao? 
  Answer: <answer>Yucheng Yang</answer>
  <confidence>unsure</confidence>
  
  Question: Who produced the Japanese battleship Mikasa? 
  Answer: <answer>Vickers</answer>
  <confidence>sure</confidence>
  
  Question: What is MaYj asap's twin city? 
  Answer: <answer>Beijing</answer>
  <confidence>unsure</confidence>

"""

rft_pararel_results = []
total_sample = len(dataset)
correct_sample = 0
un_c_match_sample = 0
Q, I, C = 0, 0, 0
for data in tqdm(dataset):
    question = data["question"]
    answer = data["answer"]
    prompt = sys_prompt + "Question: " + question +"\n"+ " Answer: "
    output_text = inference(prompt)
    model_ans = extract_content(output_text, "answer")
    model_confidence = extract_content(output_text, "confidence")
    # lines = output_text.split('\n')
    # if len(lines) == 0:
    #     lines.append("")
    # if len(lines) == 1:
    #     lines.append("")
    # model_ans = lines[0].removeprefix('<answer>').removesuffix('</answer>')
    # model_confidence = lines[1]
    # strict_eq = are_equivalent(answer, model_ans, strict_entailment=True, example=None)
    # correct = 1 if answer.lower() in model_ans.lower() or strict_eq else 0
    correct = 1 if answer.lower() in model_ans.lower() else 0
    unsure = 1 if "unsure" in model_confidence.lower() else 0
    un_c_match = 1 if ("unsure" in model_confidence.lower() and correct == 0 ) or ("sure" in model_confidence.lower() and correct == 1) else 0
    # ipdb.set_trace()
    CC, II = 0, 0
    if correct == 1 and unsure == 0:
        C += 1
        CC = 1
    if correct == 0 and unsure == 0:
        I += 1
        II = 1
    # ipdb.set_trace()
    rft_pararel_results.append({
        "question": question,
        "answer": answer,
        "model_ans": model_ans,
        "model_confidence": model_confidence,
        # "strict_eq": strict_eq,
        "correct": correct,
        "unsure": unsure,
        "un_c_match": un_c_match,
        "output_text": output_text,
        "I": II,
        "C": CC,
    })
    correct_sample += correct
    un_c_match_sample += un_c_match
D = total_sample
AED = math.sqrt(   (I * I + (D - C) * (D - C)) / (2 * D * D)   )

print(f"total_sample: {total_sample}")
print(f"correct_sample: {correct_sample}")
print(f"un_c_match_sample: {un_c_match_sample}")
print(f"accuracy: {correct_sample/total_sample}")
print(f"un_c_match_accuracy: {un_c_match_sample/total_sample}")
print(f"AED: {AED}")
rft_pararel_results.append({
    "total_sample": total_sample,
    "correct_sample": correct_sample,
    "accuracy": correct_sample/total_sample,
    "un_c_match_sample": un_c_match_sample,
    "un_c_match_accuracy": un_c_match_sample/total_sample,
    "AED": AED,
})
# save results
os.makedirs("evaluate/qwen/pararel/", exist_ok=True)
json.dump(rft_pararel_results, open("evaluate/qwen/pararel/pararel_batch10_strict_answer_origin_data_0.7temp_ID.json", "w"), indent=4)
   

100%|██████████| 5585/5585 [22:11<00:00,  4.19it/s]

total_sample: 5585
correct_sample: 1783
un_c_match_sample: 4481
accuracy: 0.3192479856759176
un_c_match_accuracy: 0.8023276633840645
AED: 0.5710288657743181


In [ ]:
import json
data_path = "dataset/ID_test_pararel.json"
# data_path = "dataset/pararel_training.json"
dataset = json.load(open(data_path))
  # You are a faithful assistant. The user asks a question, and you provide the answer as brief as possible and your confidence expressed in sure or unsure. 

sys_prompt = """
  You are a faithful assistant. The user asks a question, and you provide the answer and your confidence expressed in sure or unsure.
  Question: Who is representing The Stranglers? 
  Answer: <answer>EMI</answer>
  <confidence>sure</confidence>

  Question: Who is the president of the Shao Qiao? 
  Answer: <answer>Yucheng Yang</answer>
  <confidence>unsure</confidence>
  
  Question: Who produced the Japanese battleship Mikasa? 
  Answer: <answer>Vickers</answer>
  <confidence>sure</confidence>
  
  Question: What is MaYj asap's twin city? 
  Answer: <answer>Beijing</answer>
  <confidence>unsure</confidence>

"""
for data in dataset[:10]:
    question = data["question"]
    answer = data["answer"]
    prompt = sys_prompt + "Question: " + question +"\n"+ " Answer: "
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda:1")
    outputs = model.generate(input_ids, max_new_tokens=100)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response)
    print('-'*100)
    

In [ ]:

prompts = ["<answer>","<", "answer", ">", "</answer>", "</"]
for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    print(input_ids)
    

In [2]:
import re
def extract_content(text, tag_type="answer", model_type="default"):
    """Extract content based on tag type and model type.
    
    Args:
        text (str): The text to extract content from
        tag_type (str): Type of content to extract ("answer" or "confidence")
        model_type (str): Type of model format ("gemma" or "default")
    
    Returns:
        str: Extracted content
    """
    
    if tag_type == "answer":
        pattern = r"<answer>(.*?)</answer>"
    else:  # confidence
        pattern = r"<confidence>(.*?)</confidence>"


    matches = re.findall(pattern, text, re.DOTALL)
    return matches[0].strip() if matches else ""

In [ ]:
a = "<answer>  AD as kk usl  </answer><confidence>    a</confidence>"
answer = extract_content(a, "answer").lower().split()
confidence = extract_content(a, "confidence").lower().split()
print(answer)
print(confidence)
print('a' in answer)
print('a' in confidence)


In [ ]:
semantic_set_ids = [0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4]
semantic_set_counts = [semantic_set_ids.count(semantic_set_ids[i]) for i in range(len(semantic_set_ids))]
print(semantic_set_counts)